# Steroids or Pentoxifylline for severe Alcoholic Hepatitis (STOPAH) trial


## Generalized prognostic scores


#### Model change to prognostic scoring 

### MELD score

Model for End-Stage Liver Disease (MELD) score

\begin{equation}
\begin{array}{r}
\text { MELD }=(0.957 \times \ln (\text { creatinine }(\mathrm{mg} / \mathrm{dL})))+(0.378 \times \ln (\text { bilirubin }(\mathrm{mg} / \mathrm{dL})))+ \\
(1.120 \times \ln (\mathrm{INR}))+(0.643 \times \text { aetiology })
\end{array}
\end{equation}

A modification for large MELD Scores is 

\begin{equation}
\text { MELD-Na }=\left\{\begin{array}{ll}
\text { MELD }+1.32 \times(137-\text { sodium })-0.033 \times \text { MELD } \times(137-\text { sodium }), & \text { if MELD }>11 \\
\text { MELD }, & \text { otherwise }
\end{array}\right.
\end{equation}


When is a patient considered for a liver transplant:

1. Lille Score > 0.45

2. MELT Score 

3. Chronic Liver Failure Consortium acute-on-chronic liver failure (CLIF-C ACLF) score of ≥ 70

As these scores are linear combinations of the values, the Shapley values are simply each term. 

Our aim is to extend the named scores by generalized versions via GAMS.

In [32]:
import os
import pandas as pd
import numpy as np

#Missing value analysis 
import missingno as msno

#Models 
from sklearn.ensemble import GradientBoostingClassifier
from pygam import * 

#Analytics 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV

from sklearn.metrics import auc, roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score 


import matplotlib.pyplot as plt
import sage 
import shap

import xgboost as xgb

#!/usr/local/Cluster-Apps/python/3.11.0-icl/bin/python3.11 -m pip install ISLP

from ISLP.pygam import (plot, approx_lam, degrees_of_freedom)

In [34]:
df = pd.read_csv('/home/jlm217/rds/rds-mrc-bsu-csoP2nj6Y6Y/mimah/stopah/stopah/data/stopah.csv')

# Original MELT Score

#Compute MELT Score 

MELT = 0.957 *np.log(df['Creatinine...Merged']) + 0.378 * np.log(df['Bilirubin.Merged']) + 1.120 * np.log(df['INR...Merged.clinical.and.calc'])

p_melt = 1/(1+np.exp(-MELT))

selected = ['D28_DTH','Prednisolone']

X_MELT = ['Creatinine...Merged','Bilirubin.Merged','INR...Merged.clinical.and.calc']

df = df[X_MELT+selected]

df = df.dropna()

df0 = df[df['Prednisolone']==0].drop(['Prednisolone'],axis=1)

df = df[df['Prednisolone']==1].drop(['Prednisolone'],axis=1)

X, y = df.drop('D28_DTH', axis=1), df[['D28_DTH']]

X0, y0 = df0.drop('D28_DTH', axis=1), df0[['D28_DTH']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=0.2)

X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, random_state=1,test_size=0.2)

invalid value encountered in log


In [13]:
#Build-in hyperparameters

gam  = LogisticGAM(s(0)+s(1)+s(2)).fit(X_train.values,y_train.values)
gam0 = LogisticGAM().fit(X0_train.values,y0_train.values)

clf = GradientBoostingClassifier(n_estimators=1000, 
                                 learning_rate=0.1,
                                 max_depth=8,
                                 #min_child_weight = 1,
                                 random_state=0).fit(X_train, y_train)

#Untreated

clf0= GradientBoostingClassifier(n_estimators=1000, 
                                 learning_rate=0.1,
                                 max_depth=8, 
                                #in_child_weight = 1,
                                 random_state=0).fit(X0_train, y0_train)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [18]:
print('-----------')
print('GAM')
print('-----------')
print()
print('Untreated')
print()
print('GAM Score:'+str(gam0.score(X0_test, y0_test)))

y0_predict = gam0.predict_proba(X0_test)

auc_score0 = roc_auc_score(y0_test, y0_predict)
print('GAM AUC Score:',(auc_score0)*100)

print()
print('Treated')
print()

print('GAM Score:'+str(gam.score(X_test, y_test)))

y_predict = gam.predict_proba(X_test)

auc_score = roc_auc_score(y_test, y_predict)
print('GAM AUC Score:',(auc_score)*100)

print()
print('-----------')
print('XGB')
print('-----------')

print('Untreated')
print()
print('XGB Score:'+str(clf0.score(X0_test, y0_test)))

y0_predict = clf0.predict_proba(X0_test)

auc_score0 = roc_auc_score(y0_test, y0_predict[:,1])
print('AUC Score:',(auc_score0)*100)

print()
print('Treated')
print()

print('XGB Score:'+str(clf.score(X_test, y_test)))

y_predict = clf.predict_proba(X_test)

auc_score = roc_auc_score(y_test, y_predict[:,1])
print('AUC Score:',(auc_score)*100)

-----------
GAM
-----------

Untreated

GAM Score:0.8317757009345794
GAM AUC Score: 65.55023923444976

Treated

GAM Score:0.8691588785046729
GAM AUC Score: 78.62745098039215

-----------
XGB
-----------
Untreated

XGB Score:0.8037383177570093
AUC Score: 70.57416267942584

Treated

XGB Score:0.8037383177570093
AUC Score: 72.87581699346404


In [32]:
#First approximate lambda 

terms = [s(x, lam=0.01) for x in range(3)]

In [41]:
#lam_vals = [approx_lam(X_train.to_numpy(),terms[i],df=5) for i in range(3)]


In [7]:
gam.summary()
gam0.summary()

LogisticGAM                                                                                               
=============================================== ==========================================================
Distribution:                      BinomialDist Effective DoF:                                     14.4507
Link Function:                        LogitLink Log Likelihood:                                  -148.3156
Number of Samples:                          425 AIC:                                              325.5327
                                                AICc:                                              326.777
                                                UBRE:                                               2.7932
                                                Scale:                                                 1.0
                                                Pseudo R-Squared:                                   0.1146
Feature Function                  Lam

KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 



### Cross-validated

In [19]:
gam_cv = LogisticGAM(s(0) + s(1) + s(2)).gridsearch(X.to_numpy(), y.to_numpy())
gam_cv0 = LogisticGAM(s(0) + s(1) + s(2)).gridsearch(X0.to_numpy(), y0.to_numpy())

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--divide by zero encountered in divide
invalid value encountered in multiply
divide by zero encountered in divide
invalid value encountered in multiply
 18% (2 of 11) |####                     | Elapsed Time: 0:00:00 ETA:  00:00:00divide by zero encountered in divide
invalid value encountered in multiply
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [20]:
print('Untreated')
print()
print('GAM Score:'+str(gam_cv0.score(X0_test, y0_test)))

y0_predict_cv = gam_cv0.predict_proba(X0_test)

auc_score_cv0 = roc_auc_score(y0_test, y0_predict_cv)
print('GAM AUC Score:',(auc_score_cv0)*100)

print()
print('Treated')
print()

print('GAM Score:'+str(gam_cv.score(X_test, y_test)))

y_predict_cv = gam_cv.predict_proba(X_test)

auc_score_cv = roc_auc_score(y_test, y_predict_cv)
print('GAM AUC Score:',(auc_score_cv)*100)

Untreated

GAM Score:0.822429906542056
GAM AUC Score: 72.66746411483254

Treated

GAM Score:0.8504672897196262
GAM AUC Score: 73.79084967320262


In [21]:
gam_cv.summary()
gam_cv0.summary()

LogisticGAM                                                                                               
=============================================== ==========================================================
Distribution:                      BinomialDist Effective DoF:                                      3.1305
Link Function:                        LogitLink Log Likelihood:                                  -201.4816
Number of Samples:                          532 AIC:                                              409.2242
                                                AICc:                                             409.3046
                                                UBRE:                                               2.7739
                                                Scale:                                                 1.0
                                                Pseudo R-Squared:                                    0.061
Feature Function                  Lam

KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

